In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import os
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics.cluster import adjusted_rand_score
from sklearn.metrics.cluster import adjusted_mutual_info_score
from sklearn.metrics.cluster import homogeneity_score
import rpy2.robjects as robjects
from rpy2.robjects import pandas2ri

In [2]:
df_metrics = pd.DataFrame(columns=['ARI_Louvain','ARI_kmeans','ARI_HC',
                                   'AMI_Louvain','AMI_kmeans','AMI_HC',
                                   'Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC'])

In [3]:
workdir = './peaks_frequency_results/'
path_fm = os.path.join(workdir,'feature_matrices/')
path_clusters = os.path.join(workdir,'clusters/')
path_metrics = os.path.join(workdir,'metrics/')
os.system('mkdir -p '+path_clusters)
os.system('mkdir -p '+path_metrics)

0

In [4]:
metadata = pd.read_csv('../../input/metadata.tsv',sep='\t',index_col=0)
num_clusters = len(np.unique(metadata['label']))
print(num_clusters)

6


In [5]:
files = [x for x in os.listdir(path_fm) if x.startswith('FM')]
len(files)

11

In [6]:
files

['FM_Cusanovich20181.rds',
 'FM_Cusanovich20180.8.rds',
 'FM_Cusanovich20180.6.rds',
 'FM_Cusanovich20180.4.rds',
 'FM_Cusanovich20180.2.rds',
 'FM_Cusanovich20180.1.rds',
 'FM_Cusanovich20180.08.rds',
 'FM_Cusanovich20180.06.rds',
 'FM_Cusanovich20180.04.rds',
 'FM_Cusanovich20180.02.rds',
 'FM_Cusanovich20180.01.rds']

In [7]:
def getNClusters(adata,n_cluster,range_min=0,range_max=3,max_steps=20):
    this_step = 0
    this_min = float(range_min)
    this_max = float(range_max)
    while this_step < max_steps:
        print('step ' + str(this_step))
        this_resolution = this_min + ((this_max-this_min)/2)
        sc.tl.louvain(adata,resolution=this_resolution)
        this_clusters = adata.obs['louvain'].nunique()
        
        print('got ' + str(this_clusters) + ' at resolution ' + str(this_resolution))
        
        if this_clusters > n_cluster:
            this_max = this_resolution
        elif this_clusters < n_cluster:
            this_min = this_resolution
        else:
            return(this_resolution, adata)
        this_step += 1
    
    print('Cannot find the number of clusters')
    print('Clustering solution from last iteration is used:' + str(this_clusters) + ' at resolution ' + str(this_resolution))

In [8]:
for file in files:
    file_split = file[:-4].split('_')
    method = file_split[1]
    print(method)

    pandas2ri.activate()
    readRDS = robjects.r['readRDS']
    df_rds = readRDS(os.path.join(path_fm,file))
    fm_mat = pandas2ri.ri2py(robjects.r['data.frame'](robjects.r['as.matrix'](df_rds)))
    fm_mat.fillna(0,inplace=True)
    fm_mat.columns = metadata.index
    
    adata = sc.AnnData(fm_mat.T)
    adata.var_names_make_unique()
    adata.obs = metadata.loc[adata.obs.index,]
    df_metrics.loc[method,] = ""
    #Louvain
    sc.pp.neighbors(adata, n_neighbors=15,use_rep='X')
#     sc.tl.louvain(adata)
    getNClusters(adata,n_cluster=num_clusters)
    #kmeans
    kmeans = KMeans(n_clusters=num_clusters, random_state=2019).fit(adata.X)
    adata.obs['kmeans'] = pd.Series(kmeans.labels_,index=adata.obs.index).astype('category')
    #hierachical clustering
    hc = AgglomerativeClustering(n_clusters=num_clusters).fit(adata.X)
    adata.obs['hc'] = pd.Series(hc.labels_,index=adata.obs.index).astype('category')
    #clustering metrics
    
    #adjusted rank index
    ari_louvain = adjusted_rand_score(adata.obs['label'], adata.obs['louvain'])
    ari_kmeans = adjusted_rand_score(adata.obs['label'], adata.obs['kmeans'])
    ari_hc = adjusted_rand_score(adata.obs['label'], adata.obs['hc'])
    #adjusted mutual information
    ami_louvain = adjusted_mutual_info_score(adata.obs['label'], adata.obs['louvain'],average_method='arithmetic')
    ami_kmeans = adjusted_mutual_info_score(adata.obs['label'], adata.obs['kmeans'],average_method='arithmetic')   
    ami_hc = adjusted_mutual_info_score(adata.obs['label'], adata.obs['hc'],average_method='arithmetic')
    #homogeneity
    homo_louvain = homogeneity_score(adata.obs['label'], adata.obs['louvain'])
    homo_kmeans = homogeneity_score(adata.obs['label'], adata.obs['kmeans'])
    homo_hc = homogeneity_score(adata.obs['label'], adata.obs['hc'])

    df_metrics.loc[method,['ARI_Louvain','ARI_kmeans','ARI_HC']] = [ari_louvain,ari_kmeans,ari_hc]
    df_metrics.loc[method,['AMI_Louvain','AMI_kmeans','AMI_HC']] = [ami_louvain,ami_kmeans,ami_hc]
    df_metrics.loc[method,['Homogeneity_Louvain','Homogeneity_kmeans','Homogeneity_HC']] = [homo_louvain,homo_kmeans,homo_hc] 
    adata.obs[['louvain','kmeans','hc']].to_csv(os.path.join(path_clusters ,method + '_clusters.tsv'),sep='\t')

Cusanovich20181


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 6 at resolution 1.5
Cusanovich20180.8


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 6 at resolution 1.5
Cusanovich20180.6


...

step 0
got 7 at resolution 1.5
step 1
got 4 at resolution 0.75
step 2
got 6 at resolution 1.125
Cusanovich20180.02


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 11 at resolution 1.5
step 1
got 4 at resolution 0.75
step 2
got 6 at resolution 1.125
Cusanovich20180.01


/data/pinello/SHARED_SOFTWARE/anaconda3/envs/ATACseq_clustering/lib/python3.7/site-packages/rpy2/robjects/pandas2ri.py:191: FutureWarning: from_items is deprecated. Please use DataFrame.from_dict(dict(items), ...) instead. DataFrame.from_dict(OrderedDict(items)) may be used to preserve the key order.
  res = PandasDataFrame.from_items(items)


step 0
got 11 at resolution 1.5
step 1
got 4 at resolution 0.75
step 2
got 7 at resolution 1.125
step 3
got 4 at resolution 0.9375
step 4
got 5 at resolution 1.03125
step 5
got 7 at resolution 1.078125
step 6
got 5 at resolution 1.0546875
step 7
got 7 at resolution 1.06640625
step 8
got 5 at resolution 1.060546875
step 9
got 5 at resolution 1.0634765625
step 10
got 5 at resolution 1.06494140625
step 11
got 5 at resolution 1.065673828125
step 12
got 5 at resolution 1.0660400390625
step 13
got 7 at resolution 1.06622314453125
step 14
got 5 at resolution 1.066131591796875
step 15
got 7 at resolution 1.0661773681640625
step 16
got 7 at resolution 1.0661544799804688
step 17
got 5 at resolution 1.0661430358886719
step 18
got 7 at resolution 1.0661487579345703
step 19
got 7 at resolution 1.066145896911621
Cannot find the number of clusters
Clustering solution from last iteration is used:7 at resolution 1.066145896911621


In [9]:
df_metrics.to_csv(path_metrics+'clustering_scores.csv')

In [10]:
df_metrics

,ARI_Louvain,ARI_kmeans,ARI_HC,AMI_Louvain,AMI_kmeans,AMI_HC,Homogeneity_Louvain,Homogeneity_kmeans,Homogeneity_HC
Cusanovich20181,1,0.773607,1,1,0.900165,1,1,0.871049,1
Cusanovich20180.8,1,0.775452,1,1,0.900945,1,1,0.871049,1
Cusanovich20180.6,0.997997,0.772095,0.995993,0.997053,0.899534,0.994107,0.99707,0.871049,0.994139
Cusanovich20180.4,0.997997,0.770757,0.776624,0.997053,0.89672,0.901447,0.99707,0.868473,0.871049
Cusanovich20180.2,0.997997,0.77573,0.781572,0.997053,0.901063,0.903629,0.99707,0.871049,0.871049
Cusanovich20180.1,0.976272,0.756331,0.762181,0.971409,0.874191,0.876717,0.971565,0.84799,0.847934
Cusanovich20180.08,0.953406,0.753506,0.741634,0.950282,0.868919,0.860099,0.950569,0.843268,0.834139
Cusanovich20180.06,0.862812,0.698993,0.66422,0.883066,0.811327,0.784447,0.883664,0.791529,0.763907
Cusanovich20180.04,0.666322,0.632965,0.597908,0.756666,0.747888,0.722924,0.755172,0.730167,0.70561
Cusanovich20180.02,0.50574,0.507341,0.398319,0.610586,0.642361,0.613845,0.590883,0.631203,0.559854
